In [101]:
import pandas as pd
pd.options.display.max_columns = 100
import json, itertools
import time
import numpy as np
import matplotlib.pyplot as plt
import utils as ut
from datetime import datetime, timedelta

# Figuring out number of active sessions

In [89]:
n_words = pd.read_csv('data/n_words_clean.csv').set_index('PersonIdCode')

We build a new DataFrame where we'll store our results.

In [90]:
df = pd.DataFrame(columns=['PersonIdCode', 'FirstSession'])
df.PersonIdCode = n_words.index
df.set_index('PersonIdCode', inplace=True)
df.fillna(0, inplace=True)
df.head()

,FirstSession
PersonIdCode,
2023,0
2061,0
2129,0
2143,0
2147,0


We iterate through rows and for each member we spot his/her first active sessions, i.e. session where he/she spoke.

In [91]:
for row in n_words.iterrows():
    mmbr = row[0]
    row = row[1]
    for i,j in row.iteritems():
        if j > 0:
            df.loc[mmbr, 'FirstSession'] = i
            print(mmbr, i)
            break

2023 4802
2061 4603
2129 4601
2143 4601
2147 4601
2287 4601
2292 4601
2310 4609
2323 4601
2328 4601
2370 4601
2372 4602
2385 4601
2405 4601
2417 4901
2501 4602
2510 4602
2525 4603
2530 4602
2534 4601
2536 4602
2541 4602
2546 4605
2561 4602
2565 4606
2569 4611
2573 4614
2575 4616
2580 4620
2583 4701
2584 4703
2585 4702
2588 4705
2589 4702
2593 4701
2594 4702
2597 4703
2600 4702
2601 4702
2608 4702
2613 4701
2614 4703
2616 4703
2617 4701
2618 4707
2619 4702
2621 4702
2623 4703
2627 4702
2629 4704
2630 4702
2631 4702
2632 4702
2638 4703
2641 4701
2642 4702
2646 4709
2647 4707
2648 4706
2652 4711
2654 4712
2657 4715
2660 4716
2661 4716
2664 4717
2666 4717
2667 4718
2669 4718
2670 4802
2674 4801
2675 4802
2678 4802
2679 4802
2680 4802
2681 4801
2682 4804
2683 4801
2685 4801
2686 4804
2688 4801
2689 4804
2690 4802
2691 4801
2693 4801
2694 4801
2698 4801
2700 4801
2701 4802
2702 4801
2704 4801
2705 4802
2707 4802
2708 4802
2710 4801
2713 4802
2715 4801
2717 4802
2718 4801
2719 4801
2721 4802


In [92]:
df.head()

,FirstSession
PersonIdCode,
2023,4802
2061,4603
2129,4601
2143,4601
2147,4601


Now we find out the date of these sessions.

In [93]:
df_sesh = pd.read_csv('data/sessions.csv')

In [94]:
df['FirstSessionDate'] = 0

In [97]:
for row in df.iterrows():
    mmbr = row[0]
    row = row[1]
    sesh = row['FirstSession']
    if int(sesh) > 0:
        date = df_sesh.loc[df_sesh.ID == int(sesh)]['StartDate']
        date = date.values[0]
        date = date[:date.find('T')]
        df.loc[mmbr, 'FirstSessionDate'] = date

In [99]:
df.tail()

,FirstSession,FirstSessionDate
PersonIdCode,,
3107,5001,2015-11-30
3108,5002,2016-02-29
3109,5001,2015-11-30
3110,5001,2015-11-30
3604,4901,2011-12-05


Next, we compute members' seniority

In [102]:
today = datetime.now().date()

In [147]:
def seniority(row):
    then = row['FirstSessionDate']
    #print(then)
    if then != 0:
        then = datetime.strptime(then, '%Y-%m-%d').date()
        delta = today - then
        return int(delta.days)
    else:
        pass

In [148]:
df['Seniority'] = df.apply(seniority, axis=1)

In [150]:
df.fillna(0, inplace=True)

In [151]:
df['Seniority'] = df['Seniority'].apply(lambda x: int(x))

In [176]:
df.tail()

,FirstSession,FirstSessionDate,Seniority
PersonIdCode,,,
3107,5001,2015-11-30,395
3108,5002,2016-02-29,304
3109,5001,2015-11-30,395
3110,5001,2015-11-30,395
3604,4901,2011-12-05,1851


Adding this info to the people DataFrame

In [161]:
ppl = pd.read_csv('data/people_jonas.csv').drop('Unnamed: 0', axis=1).set_index('PersonIdCode')

In [167]:
ppl.sort_index(inplace=True)

In [170]:
False in (ppl.index == df.index)

False

In [174]:
ppl['FirstSession'] = df['FirstSession']
ppl['FirstSessionDate'] = df['FirstSessionDate']
ppl['Seniority'] = df['Seniority']

In [178]:
ppl.to_csv('data/people_jonas.csv')